In [4]:
def process_data(file_path):
    encoding_dict = {}

    with open(file_path, 'r') as file:
        lines = file.readlines()

    # Extract header for property names
    properties = lines[0].strip().split('\t')[1:]

    for line in lines[1:]:
        values = line.strip().split('\t')
        dinucleotide = values[0]  # First column is the dinucleotide
        for i, value in enumerate(values[1:], start=1):  # Start from 1 to skip the dinucleotide column
            property_name = properties[i-1]  # Adjust for zero-indexing
            if property_name not in encoding_dict:
                encoding_dict[property_name] = {}
            encoding_dict[property_name][dinucleotide] = float(value)

    return encoding_dict

# Replace 'your_data.txt' with the actual path to your dataset file
file_path = 'yourdata.txt'
encoding_dict = process_data(file_path)

# If you want to see the result for a specific property, you can print it like this:
# print(encoding_dict['conformational:Twist:1'])

# To save the dictionary to a file
import json
with open('selected_encoding_dict.json', 'w') as json_file:
    json.dump(encoding_dict, json_file, indent=4)


In [15]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.ensemble import VotingClassifier, RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier, BaggingClassifier, ExtraTreesClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression, Perceptron, SGDClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
import json

# Define classifiers
classifiers = {
    'SVM': SVC(probability=True, random_state=101),
    'Random Forest': RandomForestClassifier(n_estimators=100, random_state=101),
    'Logistic Regression': LogisticRegression(random_state=101),
    'Naive Bayes': GaussianNB(),
    'K-NN': KNeighborsClassifier(n_neighbors=5),
    'Gradient Boosting': GradientBoostingClassifier(n_estimators=100, random_state=101),
    'AdaBoost': AdaBoostClassifier(n_estimators=100, random_state=101),
    'Decision Tree': DecisionTreeClassifier(random_state=101),
    'Perceptron': Perceptron(random_state=101),
    'SGD': SGDClassifier(random_state=101),
    'Bagging': BaggingClassifier(n_estimators=100, random_state=101),
    'Extra Trees': ExtraTreesClassifier(n_estimators=100, random_state=101),
    'CatBoost': CatBoostClassifier(verbose=0, random_state=101),
    'LightGBM': LGBMClassifier(verbose=-1, random_state=101),
    'XGBoost': XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=101)
}


# Load encoding schemes file loading
all_encodings = load_encodings('encoding_dict.json')
# Function to load encoding schemes from a JSON file
def load_encodings(file_path):
    with open(file_path, 'r') as file:
        encodings = json.load(file)
    return encodings

# Function to transform a sequence into numerical data based on an encoding scheme
def transform_sequence(sequence, encoding):
    numerical_data = []
    for i in range(0, len(sequence)-1, 1):  # Assuming di-nucleotides
        di_nucleotide = sequence[i:i+2]
        numerical_data.append(encoding.get(di_nucleotide, 0))  # Default to 0 for unknown di-nucleotides
    window_avgs = [np.mean(numerical_data[i:i+15]) for i in range(0, len(numerical_data), 15)]  # Window size of 10
    return window_avgs


# Function to load and transform data from positive and negative files using an encoding scheme
def load_and_transform_data(pos_file_path, neg_file_path, encoding):
    combined_data = pd.DataFrame()
    for file_path, label in [(pos_file_path, 1), (neg_file_path, 0)]:
        with open(file_path, 'r') as file:
            sequences = file.read().split('\n')
            print (len (sequences))
        transformed_data = [transform_sequence(seq, encoding) for seq in sequences if seq]
        labels = [label] * len(transformed_data)
        df = pd.DataFrame(transformed_data)
        df['label'] = labels
        combined_data = pd.concat([combined_data, df], ignore_index=True)
    combined_data = combined_data.apply(lambda x: pd.Series(x.dropna().values)).fillna(0)  # Handle NaN
    return combined_data.drop('label', axis=1), combined_data['label']


# DataFrame to store results
results_df = pd.DataFrame(columns=['Encoding Scheme', 'Classifier', 'Accuracy', 'Precision', 'Recall', 'F1 Score'])


# Process each encoding scheme
for encoding_name, encoding in all_encodings.items():
    # Load and transform data
    X, y = load_and_transform_data('oripos.txt', 'orineg.txt', encoding)      
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=101)

    # Train and evaluate classifiers
    for name, clf in classifiers.items():
        clf.fit(X_train, y_train)
        y_pred = clf.predict(X_test)
        new_row = {
            'Encoding Scheme': encoding_name,
            'Classifier': name,
            'Accuracy': accuracy_score(y_test, y_pred),
            'Precision': precision_score(y_test, y_pred, zero_division=0),
            'Recall': recall_score(y_test, y_pred, zero_division=0),
            'F1 Score': f1_score(y_test, y_pred, zero_division=0)
        }
        results_df = pd.concat([results_df, pd.DataFrame([new_row])], ignore_index=True)
        print (encoding_name, name, accuracy_score(y_test, y_pred))

# Export results to CSV
results_df.to_csv('classification_results_cand300.csv', index=False)



254
254


C:\Users\ADMIN\AppData\Local\Temp\ipykernel_11164\2785808139.py:92: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  results_df = pd.concat([results_df, pd.DataFrame([new_row])], ignore_index=True)


conformational--Twist--1 SVM 0.5882352941176471
conformational--Twist--1 Random Forest 0.5294117647058824
conformational--Twist--1 Logistic Regression 0.5686274509803921
conformational--Twist--1 Naive Bayes 0.5686274509803921


C:\Users\ADMIN\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


conformational--Twist--1 K-NN 0.5294117647058824
conformational--Twist--1 Gradient Boosting 0.4117647058823529
conformational--Twist--1 AdaBoost 0.5490196078431373
conformational--Twist--1 Decision Tree 0.5098039215686274
conformational--Twist--1 Perceptron 0.45098039215686275
conformational--Twist--1 SGD 0.45098039215686275
conformational--Twist--1 Bagging 0.5490196078431373
conformational--Twist--1 Extra Trees 0.5882352941176471
conformational--Twist--1 CatBoost 0.47058823529411764
conformational--Twist--1 LightGBM 0.45098039215686275
conformational--Twist--1 XGBoost 0.5294117647058824
254
254
physicochemical--Stacking_energy--2 SVM 0.49019607843137253
physicochemical--Stacking_energy--2 Random Forest 0.49019607843137253
physicochemical--Stacking_energy--2 Logistic Regression 0.5294117647058824
physicochemical--Stacking_energy--2 Naive Bayes 0.45098039215686275
physicochemical--Stacking_energy--2 K-NN 0.4117647058823529
physicochemical--Stacking_energy--2 Gradient Boosting 0.54901960

KeyboardInterrupt: 

In [ ]:
### Trying to work with all

In [ ]:
import pandas as pd
import numpy as np
import json
import os
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier, BaggingClassifier, ExtraTreesClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression, Perceptron, SGDClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier

# Define classifiers
classifiers = {
    'SVM': SVC(probability=True, random_state=101),
    'Random Forest': RandomForestClassifier(n_estimators=100, random_state=101),
    'Logistic Regression': LogisticRegression(random_state=101),
    'Naive Bayes': GaussianNB(),
    'K-NN': KNeighborsClassifier(n_neighbors=5),
    'Gradient Boosting': GradientBoostingClassifier(n_estimators=100, random_state=101),
    'AdaBoost': AdaBoostClassifier(n_estimators=100, random_state=101),
    'Decision Tree': DecisionTreeClassifier(random_state=101),
    'Perceptron': Perceptron(random_state=101),
    'SGD': SGDClassifier(random_state=101),
    'Bagging': BaggingClassifier(n_estimators=100, random_state=101),
    'Extra Trees': ExtraTreesClassifier(n_estimators=100, random_state=101),
    'CatBoost': CatBoostClassifier(verbose=0, random_state=101),
    'LightGBM': LGBMClassifier(verbose=-1, random_state=101),
    'XGBoost': XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=101)
}

# Load encoding schemes file
def load_encodings(file_path):
    with open(file_path, 'r') as file:
        return json.load(file)

def transform_sequence(sequence, encoding):
    # Convert sequence into numerical data
    numerical_data = np.array([encoding.get(sequence[i:i+2], 0) for i in range(len(sequence)-1)])    
    # Calculate rolling mean using NumPy's convolve function
    window_avgs = np.convolve(numerical_data, np.ones(15)/15, mode='valid')    
    return window_avgs
    
# Function to load and transform data from positive and negative file content
def load_and_transform_data(file_content, encoding, label):
    sequences = file_content.strip().split('\n')
    transformed_data = [transform_sequence(seq, encoding) for seq in sequences if seq]
    df = pd.DataFrame(transformed_data)
    df['label'] = label
    return df

# Extract sequences from given positions
def extract_sequences(filename, start_pos, end_pos):
    with open(filename, 'r') as file:
        content = file.read()
        lines = content.split('\n')
        extracted = "\n".join(line[start_pos-1:end_pos] for line in lines if len(line) >= end_pos)
    return extracted

# Main processing function
def process_files(files, output_csv='classification_results_300win.csv'):
    all_encodings = load_encodings('selected_encoding_dict.json')
    results_df = pd.DataFrame()

    for file_name in files:
        print(f"Processing {file_name}")
        neg_content = extract_sequences(file_name, 1, 300)
        pos_content = extract_sequences(file_name, 5001, 5300)

        for encoding_name, encoding in all_encodings.items():
            pos_data = load_and_transform_data(pos_content, encoding, 1)
            neg_data = load_and_transform_data(neg_content, encoding, 0)
            combined_data = pd.concat([pos_data, neg_data], ignore_index=True)
            X, y = combined_data.drop('label', axis=1), combined_data['label']
            X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=101)

            for name, clf in classifiers.items():
                clf.fit(X_train, y_train)
                y_pred = clf.predict(X_test)
                new_row = {
                    'File': file_name,
                    'Encoding Scheme': encoding_name,
                    'Classifier': name,
                    'Accuracy': accuracy_score(y_test, y_pred),
                    'Precision': precision_score(y_test, y_pred, zero_division=0),
                    'Recall': recall_score(y_test, y_pred, zero_division=0),
                    'F1 Score': f1_score(y_test, y_pred, zero_division=0)
                }
                results_df = pd.concat([results_df, pd.DataFrame([new_row])], ignore_index=True)
                print("Results for", name, "exported to", output_csv)
    results_df.to_csv(output_csv, index=False)
    print("Results exported to", output_csv)
  # Print CSV results on the screen
    print("Results for", name, "exported to", output_csv)
    print(results_df)
    

# List of files to process
files_to_process = [    
    'Kluy_correct_10001',
    'Pichia_correct_10001',
    'Schizo_correct_10001',
    'Candiada_correct_10001',
    'Arabidopsis_correct_10001',
    'Hg19_2000_10001',
    'Sacharo_correct_10001',
    'MM8_correct_10001',
    'DM3_correct_10001'
]
process_files(files_to_process)

Processing Kluy_correct_10001
Results for SVM exported to classification_results_300win.csv
Results for Random Forest exported to classification_results_300win.csv
Results for Logistic Regression exported to classification_results_300win.csv
Results for Naive Bayes exported to classification_results_300win.csv


C:\Users\ADMIN\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Results for K-NN exported to classification_results_300win.csv
Results for Gradient Boosting exported to classification_results_300win.csv
Results for AdaBoost exported to classification_results_300win.csv
Results for Decision Tree exported to classification_results_300win.csv
Results for Perceptron exported to classification_results_300win.csv
Results for SGD exported to classification_results_300win.csv
Results for Bagging exported to classification_results_300win.csv
Results for Extra Trees exported to classification_results_300win.csv
Results for CatBoost exported to classification_results_300win.csv
Results for LightGBM exported to classification_results_300win.csv
Results for XGBoost exported to classification_results_300win.csv
Results for SVM exported to classification_results_300win.csv
Results for Random Forest exported to classification_results_300win.csv
Results for Logistic Regression exported to classification_results_300win.csv
Results for Naive Bayes exported to classif

C:\Users\ADMIN\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Results for K-NN exported to classification_results_300win.csv
Results for Gradient Boosting exported to classification_results_300win.csv
Results for AdaBoost exported to classification_results_300win.csv
Results for Decision Tree exported to classification_results_300win.csv
Results for Perceptron exported to classification_results_300win.csv
Results for SGD exported to classification_results_300win.csv
Results for Bagging exported to classification_results_300win.csv
Results for Extra Trees exported to classification_results_300win.csv
Results for CatBoost exported to classification_results_300win.csv
Results for LightGBM exported to classification_results_300win.csv
Results for XGBoost exported to classification_results_300win.csv
Results for SVM exported to classification_results_300win.csv
Results for Random Forest exported to classification_results_300win.csv
Results for Logistic Regression exported to classification_results_300win.csv
Results for Naive Bayes exported to classif

...

Results for Logistic Regression exported to classification_results_300win.csv
Results for Naive Bayes exported to classification_results_300win.csv
Results for K-NN exported to classification_results_300win.csv
Results for Gradient Boosting exported to classification_results_300win.csv
Results for AdaBoost exported to classification_results_300win.csv
Results for Decision Tree exported to classification_results_300win.csv
Results for Perceptron exported to classification_results_300win.csv
Results for SGD exported to classification_results_300win.csv
Results for Bagging exported to classification_results_300win.csv
Results for Extra Trees exported to classification_results_300win.csv
Results for CatBoost exported to classification_results_300win.csv
Results for LightGBM exported to classification_results_300win.csv
Results for XGBoost exported to classification_results_300win.csv
Results for SVM exported to classification_results_300win.csv
Results for Random Forest exported to classif

C:\Users\ADMIN\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Results for Logistic Regression exported to classification_results_300win.csv
Results for Naive Bayes exported to classification_results_300win.csv
Results for K-NN exported to classification_results_300win.csv
Results for Gradient Boosting exported to classification_results_300win.csv
Results for AdaBoost exported to classification_results_300win.csv
Results for Decision Tree exported to classification_results_300win.csv
Results for Perceptron exported to classification_results_300win.csv
Results for SGD exported to classification_results_300win.csv
Results for Bagging exported to classification_results_300win.csv
Results for Extra Trees exported to classification_results_300win.csv
Results for CatBoost exported to classification_results_300win.csv
Results for LightGBM exported to classification_results_300win.csv
Results for XGBoost exported to classification_results_300win.csv
Results for SVM exported to classification_results_300win.csv
Results for Random Forest exported to classif

C:\Users\ADMIN\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Results for Logistic Regression exported to classification_results_300win.csv
Results for Naive Bayes exported to classification_results_300win.csv
Results for K-NN exported to classification_results_300win.csv
Results for Gradient Boosting exported to classification_results_300win.csv
Results for AdaBoost exported to classification_results_300win.csv
Results for Decision Tree exported to classification_results_300win.csv
Results for Perceptron exported to classification_results_300win.csv
Results for SGD exported to classification_results_300win.csv
Results for Bagging exported to classification_results_300win.csv
Results for Extra Trees exported to classification_results_300win.csv
Results for CatBoost exported to classification_results_300win.csv
